In [4]:
import random
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from collections import Counter
from fuzzywuzzy import fuzz
import ipywidgets as widgets

mental_states = ['happy', 'sad', 'stressed', 'relaxed']
num_users = 1000

user_mental_states = [random.choice(mental_states) for _ in range(num_users)]

c:\Users\Owner\AppData\Local\Programs\Python\Python311\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [5]:
data = pd.read_csv('../datasets/data.csv')
data['imm_book_sugg'] = ''
data['imm_music_sugg'] = ''
data['imm_movie_sugg'] = ''

data.head()

,user_id,mental_state,books_preferences,music_preferences,film_preferences,imm_book_sugg,imm_music_sugg,imm_movie_sugg
0,0,happy,"['Mystery', 'Philosophy', 'Chick Lit']","['pop', 'dance', 'indie']","['Fantasy', 'Romance', 'Western']",,,
1,0,sad,"['Philosophy', 'Art', 'Graphic Novels']","['electronic', 'alternative', 'dance']","['Western', 'Comedy', 'Adventure']",,,
2,0,stressed,"['Nonfiction', 'Sports', 'Memoir']","['soul', 'metal', 'dance']","['Adventure', 'Fantasy', 'Children']",,,
3,0,relaxed,"['Thriller', 'Romance', 'Business']","['folk', 'electronic', 'pop']","['Sci-Fi', 'Children', 'Fantasy']",,,
4,1,happy,"['Science', 'Paranormal', 'Business']","['electro', 'folk', 'alternative']","['Children', 'Sci-Fi', 'Horror']",,,


In [6]:
df_books = pd.read_csv('../datasets/book_filtered_data.csv')
df_songs = pd.read_csv('../datasets/music_filtered_data.csv')
df_movies = pd.read_csv('../datasets/movie_filtered_data.csv')

In [7]:
df_books.head()

,Unnamed: 0,name,genre,emotions
0,1,Slaughterhouse-Five,"['Fiction', 'Classics', 'Science Fiction']","['Happy', 'Neutral']"
1,2,Crow Mary,['Historical Fiction'],['Angry']
2,3,"The Six Deaths of the Saint (Into Shadow, #3)","['Horror', 'Historical Fiction']","['Angry', 'Neutral']"
3,4,The Offing,"['Poetry', 'Travel']",['Angry']
4,5,The Lost Continent: Travels in Small-Town America,['Travel'],['Angry']


In [8]:
genres_for_emotions_books = df_books.groupby('emotions')['genre'].unique()
genres_for_emotions_books.head()

emotions
['Angry', 'Happy', 'Neutral']           [['Fantasy', 'Science Fiction', 'Contemporary'...
['Angry', 'Happy', 'Sad', 'Neutral']    [['Fantasy', 'Science Fiction', 'Romance', 'Fi...
['Angry', 'Neutral']                    [['Horror', 'Historical Fiction'], ['Fantasy',...
['Angry', 'Sad', 'Neutral']             [['Contemporary', 'Romance', 'Travel', 'Chick ...
['Angry']                               [['Historical Fiction'], ['Poetry', 'Travel'],...
Name: genre, dtype: object

In [9]:
books_unique = ["Fiction", "Classics", "Science Fiction", "Historical Fiction", "Horror", "Poetry", "Travel", "Nonfiction", "Comics", "Graphic Novels", "Young Adult", "Mystery", "Crime", "Thriller", "Romance", "Chick Lit", "Art", "Music", "Paranormal", "LGBT", "Children's", "Sports", "Memoir", "Biography", "Religion", "History", "Philosophy", "Self Help", "Psychology", "Business", "Spirituality", "Humor", "Science", "Business", "Religion"]
music_unique = ['alternative', 'blues', 'dance', 'electro', 'electronic', 'folk', 'indie', 'metal', 'new-age', 'pop', 'rock', 'soul', 'world-music']
movie_unique = ["Adventure", "Animation", "Children", "Comedy", "Fantasy", "Romance", "Horror", "Sci-Fi", "Western"]
  

In [10]:
genres_for_emotions_songs = df_songs.groupby('emotions')['genre'].unique()
genres_for_emotions_songs.head()

emotions
['Frustration']                                   [metal, rock]
['Happy', 'Neutral']                              [world-music]
['Happy']               [dance, electro, electronic, pop, soul]
['Neutral']                                     [folk, new-age]
['Sad']                             [alternative, blues, indie]
Name: genre, dtype: object

In [11]:
df_movies['genre'].unique()

array(["['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy']",
       "['Adventure', 'Children', 'Fantasy']", "['Comedy', 'Romance']",
       ..., "['Action', 'Adventure', 'Animation', 'Fantasy', 'Horror']",
       "['Animation', 'Children', 'Comedy', 'Fantasy', 'Sci-Fi']",
       "['Animation', 'Children', 'Comedy', 'Western']"], dtype=object)

In [12]:
genres_for_emotions_movies = df_movies.groupby('emotions')['genre'].unique()
genres_for_emotions_movies.head()

emotions
['Frustration', 'Happy', 'Neutral']           [['Action', 'Adventure', 'Fantasy'], ['Action'...
['Frustration', 'Happy', 'Sad', 'Neutral']    [['Action', 'Adventure', 'Romance'], ['Action'...
['Frustration', 'Neutral']                    [['Adventure', 'Children', 'Fantasy'], ['Fanta...
['Frustration', 'Sad', 'Neutral']             [['Adventure', 'Animation', 'Children', 'Comed...
['Frustration']                               [['Animation', 'Children', 'Fantasy', 'Musical...
Name: genre, dtype: object

In [13]:
def get_state(df, user_id, mental_state):
  user_row = df[(df["user_id"] == user_id) & (df["mental_state"] == mental_state)]

  if not user_row.empty:
        user_row = user_row.iloc[0]
        books_preferences = user_row["books_preferences"]
        films_preferences = user_row["film_preferences"]
        music_preferences = user_row["music_preferences"]
  else:
        books_preferences = []
        films_preferences = []
        music_preferences = []


  return {'user_id': user_id, 'mental_state': mental_state, 'books': books_preferences, 'films': films_preferences, 'music': music_preferences}
     


In [14]:
def get_user_pref_network(df, user_id):
  user_row = df[(df["user_id"] == user_id)]

  if not user_row.empty:
        user_row = user_row.iloc[0]
        books_preferences = user_row["books_preferences"]
        films_preferences = user_row["film_preferences"]
        music_preferences = user_row["music_preferences"]
  else:
        books_preferences = []
        films_preferences = []
        music_preferences = []


  return books_preferences, films_preferences, music_preferences
     

In [15]:
def get_matching_value(book, music, film, user_id, data):

  book_1, film_1, music_1 = get_user_pref_network(data, user_id)
  book_score = fuzz.token_set_ratio(book, book_1)
  music_score = fuzz.token_set_ratio(music, music_1)
  film_score = fuzz.token_set_ratio(film, film_1)

  return (book_score + music_score + film_score)/3

In [16]:
def get_similar_users(data, user_id, mental_state):

  similar_users = []

  for idx, row in data.iterrows():
    if row['mental_state'] == mental_state:
      matching_value = get_matching_value(row['books_preferences'], row['music_preferences'], row['film_preferences'], user_id, data)
      if matching_value > 85:
        similar_users.append(user_id)


  return similar_users

In [17]:
def get_ego_suggestions(similar_users_high, data, mental_state):

  book_suggestions = []
  music_suggestions = []
  film_suggestions = []

  for user_id in similar_users_high:
    user_row = data[(data["user_id"] == user_id) & (data["mental_state"] == mental_state)]
    user_row = user_row.iloc[0]
    book_suggestions.append(user_row['imm_book_sugg'])
    music_suggestions.append(user_row['imm_music_sugg'])
    film_suggestions.append(user_row['imm_movie_sugg'])

  return book_suggestions, music_suggestions, film_suggestions

In [18]:
def get_suggestions_from_user_pref(user_id, book_data, film_data, music_data, mental_state, data):

  max_elements = 100

  state_id = get_state(data, user_id, mental_state)
  book_preferences = state_id['books']
  music_preferences = state_id['music']
  film_preferences = state_id['films']

  books = book_data.loc[book_data['genre'].apply(lambda x: any(g in x for g in book_preferences)), 'name'].tolist()[:max_elements]
  music = music_data.loc[music_data['genre'].apply(lambda x: any(g in x for g in music_preferences)), 'name'].tolist()[:max_elements]
  film = film_data.loc[film_data['genre'].apply(lambda x: any(g in x for g in film_preferences)), 'name'].tolist()[:max_elements]

  return books, music, film

In [19]:
def get_suggestions_from_explore(user_id, book_data, film_data, music_data, mental_state, data):

  max_elements = 100

  state_id = get_state(data, user_id, mental_state)

  exp_book_pref = [item for item in books_unique if item not in state_id['books']]
  exp_music_pref = [item for item in music_unique if item not in state_id['music']]
  exp_film_pref = [item for item in movie_unique if item not in state_id['films']]

  books = book_data.loc[book_data['genre'].apply(lambda x: any(g in x for g in exp_book_pref)), 'name'].tolist()[:max_elements]
  music = music_data.loc[music_data['genre'].apply(lambda x: any(g in x for g in exp_music_pref)), 'name'].tolist()[:max_elements]
  film = film_data.loc[film_data['genre'].apply(lambda x: any(g in x for g in exp_film_pref)), 'name'].tolist()[:max_elements]

  return books, music, film

In [20]:

def get_expl_suggest(ego, usr, ratio):
  suggestion = []

  if ego == ['']:
     suggestion = random.sample(usr, ratio)
  elif len(ego) < ratio:
    suggetion = random.sample(ego, len(ego))
    if len(usr) >= (ratio - len(ego)):
      suggestion += random.sample(usr, ratio - len(ego))
    else:
      suggestion += random.sample(usr, len(usr))
  else:
    suggestion = random.sample(ego, ratio)

  return suggestion

In [21]:
def get_reward(feedback):

    if feedback == "good":
        return 6
    elif feedback == "bad":
        return -6
    elif feedback == "skip":
        return -2
    else:
        return 0

In [22]:
def update_ratio(ratio, reward, selection):
    actions = ['book', 'music', 'films']
    unselected = [action for action in actions if action != selection]

    alpha = 0.1  # Step size for updating ratios
    if not selection == '':
      ratio[selection] = ratio[selection] + 2 * alpha * reward
      print(ratio[selection])
      ratio[selection] = round(ratio[selection])

    for action in unselected:
      ratio[action] = round(ratio[action] - alpha * reward)

    return ratio

In [23]:
def get_selected_action(selection, books, music, films):
  if selection in books:
    return 'book'
  elif selection in music:
    return 'music'
  else:
    return 'films'

In [24]:
def get_selection():

  break_val = 0
  try_another = 0
  selected_item = ''
  feedback = ''

  print('###################')
  print('Types of selections')
  print()
  print('Exit: E | Try Anothet: C | Select given: S')
  print('###################')
  selection = input('Enter your selection : ')

  if selection == 'e':
    break_val = 1
  elif selection == 'c':
    try_another = 1
  elif selection == 's':
    print('######################################')
    selected_item = input('Enter selected item : ')
    print('######################################')
    feedback = input('feedback |(good or bad)|: ')

  return {'break': break_val, 'skip': try_another, 'item': selected_item, 'feedback': feedback}
 

In [25]:
def get_column_name(action):

  if action == 'book':
    return 'imm_book_sugg'
  elif action == 'music':
    return 'imm_music_sugg'
  else:
    return 'imm_film_sugg'

In [26]:

def add_imm_suggestion(user_id, action, selected_item):

  column = get_column_name(action)
  data.loc[data['user_id'] == 1, column] = selected_item

In [27]:
def update_preferences(user_id, column, selected_item):

  if action == 'book':
    data = df_books
  elif action == 'music':
    data = df_songs
  else:
    data = df_movies

  item = data[data['name'] == selected_item]

  if not item.empty:
    prefernces = data['genre'].values[0]

  else:
    None

In [30]:
print('##############################')
user_id = input('Enter your user_id: ')
print('##############################')
mental_state = input('Enter your mental_state: ')
ratio = {'book': 3, 'music': 3, 'films': 3}
number_of_episoids = 20
rewards = []
explore = 0
feedback = ''
action = ''

while number_of_episoids>0:

  epsilon = 0.2

  state_id = get_state(data, int(user_id), mental_state)
  similar_users = get_similar_users(data, int(user_id), mental_state)

  # actions from ego
  book_ego, music_ego, film_ego = get_ego_suggestions(similar_users, data, mental_state)

  # actions from user preferences
  book_usr, music_usr, film_usr = get_suggestions_from_user_pref(int(user_id), df_books, df_movies, df_songs, mental_state, data)

  # actions for exploration
  book_exp, music_exp, film_exp = get_suggestions_from_explore(user_id, df_books, df_movies, df_songs, mental_state, data)

  # initial_ratios for the dynamic ratio adjusting
  book_ratio = ratio['book']
  music_ratio = ratio['music']
  film_ratio = ratio['films']


  if np.random.rand() < epsilon:
    explore = 1
    print('Exploration results')
    book_suggestion = random.sample(book_exp, book_ratio)
    film_suggestion = random.sample(film_exp, film_ratio)
    music_suggestion = random.sample(music_exp, music_ratio)

  else:
    print('Exploitation results')
    book_suggestion = get_expl_suggest(book_ego, book_usr, book_ratio)
    film_suggestion = get_expl_suggest(film_ego, film_usr, film_ratio)
    music_suggestion = get_expl_suggest(music_ego, music_usr, music_ratio)


  print(f'Book suggestions are : {book_suggestion}')
  print(f'music_suggestions are: {music_suggestion}')
  print(f'movie suggestions are: {film_suggestion}')

  selection = get_selection()
  if selection['break'] == 1:
    break

  elif selection['skip'] == 1:
    feedback = 'skip'
    reward = get_reward(feedback)
    rewards.append(reward)
  else:
    selected_item = selection['item']
    feedback = selection['feedback']
    action = get_selected_action(selected_item, book_suggestion, music_suggestion, film_suggestion)
    add_imm_suggestion(user_id, action, selected_item)
    ratio = update_ratio(ratio, reward, action)

    if explore == 1:
      update_preferences(user_id, action, selected_item)


  print('#####################################################')
  print(f'Updated ratios - {ratio}')

  print(rewards)
  number_of_episoids = number_of_episoids - 1

plt.figure(figsize=(10, 6))
plt.plot(number_of_episoids, rewards, marker='o', linestyle='-', color='b', label='Reward')
plt.xlabel('Number of Trials')
plt.ylabel('Reward')
plt.title('Rewards vs Number of Trials')
plt.legend()
plt.grid(True)
plt.show()



##############################
##############################
Exploration results
Book suggestions are : ['The Retreat (Detective Elin Warner, #2)', 'Shadow and Bone (The Shadow and Bone Trilogy, #1)', 'I Have Some Questions for You']
music_suggestions are: ['Salvador', 'Daddy Issues', 'Numb']
movie suggestions are: ['Happiness Is in the Field (Bonheur est dans le pré, Le) (1995)', 'Powder (1995)', 'Steal Big, Steal Little (1995)']
###################
Types of selections

Exit: E | Try Anothet: C | Select given: S
###################


NameError: name 'reward' is not defined

In [31]:
from collections import defaultdict
import random

# Initialize Q-table
Q = defaultdict(lambda: defaultdict(float))

# Define parameters
alpha = 0.1  # Learning rate
gamma = 0.9  # Discount factor
epsilon = 0.2  # Exploration rate
num_episodes = 20

# Reward mapping
def get_reward(feedback):
    if feedback == "good":
        return 6
    elif feedback == "bad":
        return -6
    elif feedback == "skip":
        return -2
    else:
        return 0

# Function to choose an action using epsilon-greedy policy
def choose_action(state, actions):
    if np.random.rand() < epsilon:
        return random.choice(actions)
    else:
        q_values = [Q[state][action] for action in actions]
        max_q = max(q_values)
        max_actions = [actions[i] for i, q in enumerate(q_values) if q == max_q]
        return random.choice(max_actions)

# Extract state for Q-learning
def get_state(user_id, mental_state):
    return f"user_{user_id}_state_{mental_state}"

# Function to update the Q-table
def update_q_table(state, action, reward, next_state, next_actions):
    best_next_action = max(Q[next_state], key=Q[next_state].get, default=None)
    td_target = reward + gamma * Q[next_state][best_next_action] if best_next_action else reward
    Q[state][action] += alpha * (td_target - Q[state][action])

# Run Q-learning
rewards = []
for episode in range(num_episodes):
    user_id = random.choice(range(num_users))
    mental_state = random.choice(mental_states)
    state = get_state(user_id, mental_state)
    
    # Get recommendations
    similar_users = get_similar_users(data, user_id, mental_state)
    book_ego, music_ego, film_ego = get_ego_suggestions(similar_users, data, mental_state)
    book_usr, music_usr, film_usr = get_suggestions_from_user_pref(user_id, df_books, df_movies, df_songs, mental_state, data)
    book_exp, music_exp, film_exp = get_suggestions_from_explore(user_id, df_books, df_movies, df_songs, mental_state, data)
    
    # Combine actions
    actions = ['book', 'music', 'film']
    book_suggestion = random.sample(book_exp, 3)
    film_suggestion = random.sample(film_exp, 3)
    music_suggestion = random.sample(music_exp, 3)
    
    # Choose action using epsilon-greedy policy
    action = choose_action(state, actions)
    
    # Get feedback and calculate reward
    selection = get_selection()
    if selection['break'] == 1:
        break
    elif selection['skip'] == 1:
        feedback = 'skip'
    else:
        selected_item = selection['item']
        feedback = selection['feedback']
        action = get_selected_action(selected_item, book_suggestion, music_suggestion, film_suggestion)
    
    reward = get_reward(feedback)
    rewards.append(reward)
    
    # Get next state (assuming user state may change; here it is static for simplicity)
    next_state = state
    next_actions = actions
    
    # Update Q-table
    update_q_table(state, action, reward, next_state, next_actions)
    
    print(f"Episode {episode + 1}: State={state}, Action={action}, Reward={reward}")

# Plot rewards
plt.figure(figsize=(10, 6))
plt.plot(range(num_episodes), rewards, marker='o', linestyle='-', color='b', label='Reward')
plt.xlabel('Number of Episodes')
plt.ylabel('Reward')
plt.title('Rewards vs Number of Episodes')
plt.legend()
plt.grid(True)
plt.show()


TypeError: get_state() takes 2 positional arguments but 3 were given